In [48]:
from newspaper import Article
import nltk
import os
import pandas as pd
import numpy as npg
nltk.download('punkt')
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
import csv

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] =os.getcwd()+'\google_cloud_api.json'

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tjbli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [49]:
westwing = pd.read_csv("../data/west_wing_reads.csv")
otherData = pd.read_csv("../data/articles.csv")

In [50]:
websitesToKeep = ["foxnews.com", "washingtonexaminer.com", "wsj.com", "washingtontimes.com", "usatoday.com", 
                  "washingtonpost.com", "nytimes.com", "cnn.com", "nbcnews.com", "thehill.com"]

In [51]:
westwing = westwing[westwing['publisher'].isin(websitesToKeep)]
westwing = westwing.reset_index()

In [52]:
westwing = otherData

In [53]:
df_sentiment = {"Publisher": np.empty([1,len(westwing)], dtype = object), 
                                       "Score": np.zeros(len(westwing)),
                                       "Magnitude": np.zeros(len(westwing)),
                                        "Keywords": np.empty([1,len(westwing)], dtype=object)}

x = np.arange(0, len(westwing))
client = language.LanguageServiceClient()

for i in x:
    url = westwing['link'][i]
    article1 = Article(url)
    article1.download()
    article1.parse()
    article1.nlp()
    text = article1.summary
    
    document = types.Document(
    content=text,
    type=enums.Document.Type.PLAIN_TEXT)

    # Detects the sentiment of the text
    sentiment = client.analyze_sentiment(document=document).document_sentiment

    df_sentiment["Publisher"][0][i] = westwing["publisher"][i]
    df_sentiment["Score"][i] = sentiment.score 
    df_sentiment["Magnitude"][i] = sentiment.magnitude
    
    keyword = ''
    for j in np.arange(0,len(article1.keywords)):
        keyword = keyword + str(article1.keywords[j]) + ' '
       
    df_sentiment["Keywords"][0][i] = keyword
    print(i)

DefaultCredentialsError: File C:\Users\tjbli\Documents\GitHub\cdss-hackathon-2018\src\google_cloud_api.json was not found.

In [54]:
with open('dict.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in df_sentiment.items():
       writer.writerow([key, value])